Make appropriate changes in this code as per your classifier and return me this file. It should be executable with your test data.

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm

import matplotlib.pyplot as plt # For data viz
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

print('System Version:', sys.version)
print('PyTorch version', torch.__version__)
print('Torchvision version', torchvision.__version__)
print('Numpy version', np.__version__)
print('Pandas version', pd.__version__)

System Version: 3.11.4 | packaged by Anaconda, Inc. | (main, Jul  5 2023, 13:38:37) [MSC v.1916 64 bit (AMD64)]
PyTorch version 2.1.0+cpu
Torchvision version 0.16.0+cpu
Numpy version 1.26.0
Pandas version 1.5.3


#### Roll No: M003

In [9]:
test_dir = 'train_validate_test/test'

####  Replace with your own class and transform. Just provide me the test loader

In [10]:
class OcularDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return self.data[idx]
    @property
    def classes(self):
        return self.data.classes

    
#DatLoader
transform = transforms.Compose([
    transforms.Resize((128, 128)),    
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# trainset = OcularDataset(train_dir, transform)
testset = OcularDataset(test_dir, transform)

# validateset = OcularDataset(validate_dir, transform)


# trainloader = DataLoader(trainset, batch_size=16, shuffle=True)
testloader = DataLoader(testset, batch_size=16, shuffle=True)
# validateloader = DataLoader(validateset, batch_size=16, shuffle=True)

#### Your classifier class

In [11]:
class OcularClassifer(nn.Module):
    def __init__(self, num_classes=6):
        super(OcularClassifer, self).__init__()
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        enet_out_size = 1280
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(enet_out_size, num_classes)
        )
    def forward(self, x):
        # Connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output

In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'trained_efficientnet1.pth' # your saved model
model = OcularClassifer(num_classes=6).to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [13]:
from sklearn.metrics import accuracy_score, classification_report
# Testing
model.eval()
all_test_preds = []
all_test_labels = []
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_test_preds.extend(predicted.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

# Calculate test accuracy
test_accuracy = accuracy_score(all_test_labels, all_test_preds)
print(f"Test Accuracy: {test_accuracy}")

Test Accuracy: 0.5509761388286334


In [14]:
# Generate a classification report
class_names = testset.classes
report = classification_report(all_test_labels, all_test_preds, target_names=class_names)
print(report)

                                  precision    recall  f1-score   support

age_related_macular_degeneration       0.54      0.60      0.57        78
                        cataract       0.94      0.82      0.88        79
                        diabetes       0.33      0.22      0.26        73
                        glaucoma       0.48      0.57      0.52        76
                    hypertension       0.82      0.55      0.66        75
                          normal       0.36      0.53      0.43        80

                        accuracy                           0.55       461
                       macro avg       0.58      0.55      0.55       461
                    weighted avg       0.58      0.55      0.55       461

